In [37]:
# Importing relevant packages 
#!pip3 install python-Levenshtein --user
#!pip3 install fuzzywuzzy --user

import fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [38]:
# Cleaning the prepared NETS data set 
df1 = pd.read_stata('H:/InnovationProject/data/temp/NETS_prep.dta') 
df1['hqcompany'] = df1['hqcompany'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
df1['hqtradename'] = df1['hqtradename'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

In [39]:
# Importing information to match counties to commuting zones 
df4 = pd.read_stata('H:/InnovationProject/data/raw/var_CommutingZones/CZ_combined.dta')
df4 = df4[['county_fips', 'CZ_depagri_1990']]

length1 = 5
df4['county_fips'] = df4['county_fips'].astype(str).str.zfill(length1)

In [40]:
# Cleaning the inventor data set, only keeping the necessary information and merging in commuting zones 
df2 = pd.read_stata('H:/InnovationProject/data/temp/patentdata_clean_assignee.dta')

length1 = 2
length2 = 3

# Fill var1 and var2 with leading zeros and combine them
df2['combined_var'] = df2['state_fips_inventor'].astype(str).str.zfill(length1) + df2['county_fips_inventor'].astype(str).str.zfill(length2)
df2 = df2.rename(columns={'combined_var': 'county_fips'})
df2 =df2[['assignee_id', 'assignee_std', 'gvkey', 'county_fips']]

df2 = pd.merge(df4, df2, on=['county_fips'])

df2 = df2[['assignee_id', 'assignee_std', 'CZ_depagri_1990']]
df2 = df2.drop_duplicates(subset=['assignee_id', 'CZ_depagri_1990'])
df2['assignee_std'] = df2['assignee_std'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

In [48]:
# Merging both data sets together based on direct matching 
# Writing the function for fuzzy matching
def match_names(row, df2_filtered, threshold=90):
    name = row['hqcompany']
    matches = process.extractOne(name, df2_filtered['hqcompany'], scorer=fuzz.token_sort_ratio)
    if matches[1] >= threshold:
        return matches[0]
    return None

for value in df2['CZ_depagri_1990'].unique():     
    print(value)
    df2_subset = df2[df2['CZ_depagri_1990'] == value]
    df1_subset = df1[df1['CZ_depagri_1990'] == value ]
    df2_subset = df2_subset.rename(columns={'assignee_std': 'hqcompany'})
    matched1 = pd.merge(df1_subset, df2_subset, on=['hqcompany', 'CZ_depagri_1990'], how='inner')
    
    # Identify direct matches and delete them; for both data sets
    df1_filtered = df1_subset.merge(matched1, on=['hqcompany'], how='left', indicator=True)
    df1_filtered = df1_filtered[df1_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    df2_filtered = df2_subset.merge(matched1, on=['hqcompany'], how='left', indicator=True)
    df2_filtered = df2_filtered[df2_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    # Renaming the matched variables  
    df2_filtered = df2_filtered[['assignee_id_x', 'hqcompany', 'CZ_depagri_1990_x']]
    df2_filtered = df2_filtered.rename(columns={'assignee_id_x': 'assignee_id'})
    df2_filtered = df2_filtered.rename(columns={'CZ_depagri_1990_x': 'CZ_depagri_1990'})
    
    df1_filtered = df1_filtered[['dunsnumber_x', 'hqduns_x', 'hqcompany', 'hqtradename_x', 'CZ_depagri_1990_x']]
    df1_filtered = df1_filtered.rename(columns={'dunsnumber_x': 'dunsnumber'})
    df1_filtered = df1_filtered.rename(columns={'hqtradename_x': 'hqtradename'})
    df1_filtered = df1_filtered.rename(columns={'hqduns_x': 'hqduns'})
    df1_filtered = df1_filtered.rename(columns={'CZ_depagri_1990_x': 'CZ_depagri_1990'})
    
    # Step 2: Fuzzy Matching
    df1_filtered['matched_name'] = df1_filtered.apply(match_names, axis=1, df2_filtered=df2_filtered)
    df1_filtered = df1_filtered.rename(columns={'hqcompany': 'original_name'})
    df1_filtered = df1_filtered.rename(columns={'matched_name': 'hqcompany'})
    df1_filtered = df1_filtered.dropna(subset=['hqcompany'])
    matched2 = pd.merge(df2_filtered, df1_filtered, on=['hqcompany', 'CZ_depagri_1990'], how='inner')
    
    df_combined = pd.concat([matched1, matched2], ignore_index=True)
    
    # Define, dynamic file path 
    file_path = f"H:/InnovationProject/data/temp/matching_NETS/match_{value}.csv"
    df_combined.to_csv(file_path) 
    
    df1_cleaned = df1_filtered.applymap(lambda x: x.encode('latin-1', 'replace').decode('latin-1') if isinstance(x, str) else x)
    df2_cleaned = df2_filtered.applymap(lambda x: x.encode('latin-1', 'replace').decode('latin-1') if isinstance(x, str) else x)
    
    file_path = f"H:/InnovationProject/data/temp/matching_NETS/nonmatch_NETS_{value}.csv"
    df1_cleaned.to_stata(file_path) 
    
    file_path = f"H:/InnovationProject/data/temp/matching_NETS/nonmatch_patents_{value}.csv"
    df2_cleaned.to_stata(file_path) 

19600.0
H:/InnovationProject/data/temp/matching_NETS/match_19600.0.csv
H:/InnovationProject/data/temp/matching_NETS/nonmatch_NETS_19600.0.csv
H:/InnovationProject/data/temp/matching_NETS/nonmatch_patents_19600.0.csv
